In [ ]:
import os
import pandas as pd
import sqlite3

print(os.getcwd())  # Shows your current working directory
print(os.path.exists('/content/books.csv'))  # Checks if the file exists

/content
True


In [ ]:
print('Extract data....')
df = pd.read_csv(r'/content/books.csv')
print(f"Found {len(df)} rows")

print('Cleaning data....')

Extract data....
Found 5 rows
Cleaning data....


In [ ]:
# Clean title - remove extra spaces
df['title'] = df['title'].str.strip()


In [ ]:
# Fix publish_year: convert to integer, set 0 if invalid/missing
df['publish_year'] = pd.to_numeric(df['publish_year'], errors='coerce')


In [ ]:
# Fix price: NMust be positive
df['price'] = pd.to_numeric(df['price'], errors='coerce').clip(lower=0)

In [ ]:
# Drop row with missing Id
df = df.dropna(subset=['id'])
print(df)

   id                  title  publish_year  price
0   1           Harry Potter        2001.0  29.99
1   2          Lord of Rings        1995.0  35.00
2   3  To Kill a Mockingbird           NaN  35.00
3   4           Harry Potter        2001.0  29.99
4   5                    NaN        1984.0  20.00


In [ ]:
# Load database
print('Load data....')
conn = sqlite3.connect('library.db')
cursor = conn.cursor()


Load data....


In [ ]:
# Create table
cursor.execute('''CREATE TABLE IF NOT EXISTS books
                  (id INTEGER PRIMARY KEY,
                  title TEXT, author TEXT,
                  publish_year INTEGER, price REAL)''')


In [ ]:
df.to_sql('book', conn, if_exists='replace', index=False)
conn.commit()


In [ ]:
# Validate
print('Validating....')
cursor.execute("SELECT COUNT(*) FROM books")
db_count = cursor.fetchall()[0]
print(f'Database has: {db_count} rows')


Validating....
Database has: (5,) rows


In [ ]:
# Check for duplicate
cursor.execute("SELECT id, COUNT(*) FROM books GROUP BY id HAVING COUNT(*) > 1")
duplicate = cursor.fetchall()
if duplicate:
  print('Error: Duplicate ID found')
else:
  print('No duplicate ID found')


No duplicate ID found


In [ ]:
# Sample data check
cursor.execute('SELECT SUM(price) FROM books')
db_sum = cursor.fetchone()[0]
csv_sum = df['price'].sum()
print(f'price sum: CSV={csv_sum}, DB{db_sum}')
conn.close()
print('Migration complete!')

price sum: CSV=149.98, DB149.98
Migration complete!
